## Imports

Make sure to import all of the necessary dependencies for your model to operate. Both pipeline 1 and 2 should include the following:

1. utils.py
2. Path to the child model class (in this case, BERT_NER)
3. Any extra dependencies unrelated to the class (in this case, BERT's AutoTokenizer)

In [1]:
from BERT_NER import *
from utils import *
from transformers import AutoTokenizer

c:\Users\Abdelaal\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


## Instantiating the Modules

First, store the path to the test dataset

In [2]:
test_data_dir = "dataset/PIZZA_test.json"

Then, load modules 1 and 2 using the classes you created. You can change the constructor to take any parameters; just make sure that you classes contain an implementation of the "predict_labels" method and takes no extra parameters.

In [3]:
tokenizer = AutoTokenizer.from_pretrained("models/pizza_tokenizer")
module1 = BERT_NER('models/ISA_model_2', tokenizer)
module2 = BERT_NER('models/order_parser_improved_2', tokenizer)

Done! The rest of the code should not be changed

In [5]:
characters_to_remove = "!@#$%^&*()_+=[]{}|;:\",.<>?/`~"
sentence = input("Please enter your order: ")
sentence = ''.join(char for char in sentence if char not in characters_to_remove)

# Segment orders
isa_words, isa_labels = module1.predict_labels(sentence)
orders = segment_orders(isa_words, isa_labels)
top_decoupled = ''
for order_pair in orders:
    order = order_pair[0]
    is_pizza = order_pair[1]
    parser_words, parser_labels = module2.predict_labels(order)

    if is_pizza: id = 0
    else: id = 1

    top_decoupled += (generate_top_decoupled(order, parser_labels, id) + ' ')

print("Input Order: " + sentence)
print("-------------------------------------------------")
print("Top Decoupled: " + top_decoupled[:-1])
print("-------------------------------------------------")
print(parse_and_convert_to_json(top_decoupled[:-1]))

Input Order: I would like to order two large chicago style pizzas with extra cheese pepperoni  no onions and bacon  and three small thin crust pizzas with a lot of olives and ham I also want five large bottles of coke and two medium cans of diet pepsi
-------------------------------------------------
Top Decoupled: (PIZZAORDER (NUMBER two ) (SIZE large ) (STYLE chicago style ) (COMPLEX_TOPPING (QUANTITY extra ) (TOPPING cheese ) ) (TOPPING pepperoni ) (NOT (TOPPING onions ) ) (TOPPING bacon ) ) (PIZZAORDER (NUMBER three ) (SIZE small ) (STYLE thin crust ) (COMPLEX_TOPPING (QUANTITY a lot of ) (TOPPING olives ) ) (TOPPING ham ) ) (DRINKORDER (NUMBER five ) (SIZE large ) (CONTAINERTYPE bottles ) (DRINKTYPE coke ) ) (DRINKORDER (NUMBER two ) (SIZE medium ) (CONTAINERTYPE cans ) (DRINKTYPE diet pepsi ) )
-------------------------------------------------
{
  "ORDER": {
    "PIZZAORDER": [
      {
        "NUMBER": "two",
        "SIZE": "large",
        "STYLE": [
          {
            "N

In [5]:
import json
correct_count = 0
i = 0
with open(test_data_dir, 'r') as infile:
    for line in infile:
        instance = json.loads(line)
        input_sentence = instance.get(f"test.SRC", "")
        input_sentence = ''.join(char for char in input_sentence if char not in characters_to_remove)
        top = instance.get(f"test.TOP", "")
        true_top_decoupled = generate_top_decoupled_from_top(top)

        isa_words, isa_labels = module1.predict_labels(input_sentence)
        orders = segment_orders(isa_words, isa_labels)
        top_decoupled = ''
        for order_pair in orders:
            order = order_pair[0]
            is_pizza = order_pair[1]
            parser_words, parser_labels = module2.predict_labels(order)

            if is_pizza: id = 0
            else: id = 1

            top_decoupled += (generate_top_decoupled(order, parser_labels, id) + ' ')
        
        top_decoupled = "(ORDER " + top_decoupled + ')'

        if top_decoupled == true_top_decoupled: correct_count += 1
        else:
            print(f"Row Index: {i}")
            print("Input Sentence: " + input_sentence)
            print("True Output: " + true_top_decoupled)
            print("Predicted Output: " + top_decoupled)
            print("----------------------------------------------------------------------")
        

        i += 1

Row Index: 1129
Input Sentence: can i have a large pizza with red onions bell peppers pepperoni and lettuce
True Output: (ORDER (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING red onions ) (TOPPING bell peppers ) (TOPPING pepperoni ) (TOPPING lettuce ) ) )
Predicted Output: (ORDER (PIZZAORDER (NUMBER a ) (SIZE large ) (TOPPING red onions ) (TOPPING bell ) (TOPPING peppers ) (TOPPING pepperoni ) (TOPPING lettuce ) ) )
----------------------------------------------------------------------
Row Index: 1176
Input Sentence: two small double cheese double pepperoni pizzas please
True Output: (ORDER (PIZZAORDER (NUMBER two ) (SIZE small ) (COMPLEX_TOPPING (QUANTITY double ) (TOPPING cheese ) ) (COMPLEX_TOPPING (QUANTITY double ) (TOPPING pepperoni ) ) ) )
Predicted Output: (ORDER (PIZZAORDER (NUMBER two ) (SIZE small ) (COMPLEX_TOPPING (QUANTITY double ) (TOPPING cheese ) (QUANTITY double ) (TOPPING pepperoni ) ) ) )
----------------------------------------------------------------------


In [6]:
correct_count / i

0.9985261606484893